In [1]:

import os
from openai import OpenAI
from dotenv import load_dotenv
from groq import Groq
import json
load_dotenv()

# OPENAI version requires a paid account, and you will have to switch out model
# client = OpenAI(
#     api_key=os.environ["OPENAI_API_KEY"]
# )
# GROq version free
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)



In [2]:
# system_prompt_iterations is a directory of textfiles of just system prompts

In [170]:
import logging

logger = logging.getLogger()
logging.basicConfig(filename='myapp.log', level=logging.INFO)
class FunctionSpecWriter():

    tools = [
    {
      "type": "function",
      "function": {
        "name": "get_function_within_code",
        "description": "Get the function spec for a function that is called in the function being analyzed for which we can't find a function spec",
        "parameters": {
          "type": "object",
          "properties": {
            "function_name": {
              "type": "string",
              "description": "The name of the function whose spec is required to complete the task"
            },
          },
          "required": ["function_name"]
        }
      }
    },
    {
      "type": "function",
      "function": {
        "name": "get_function_within_package",
        "description": "Gets the function spec of a function in a well documented function using the internet",
        "parameters": {
          "type": "object",
          "properties": {
            "function_name": {
              "type": "string",
              "description": "The name of the function whose spec is required to complete the task"
            },
          },
          "required": ["function_name"]
        }
      }
    }
  ]

    def __init__(self,system_message,model,max_recursive_depth=10):
        self.system_message = system_message
        self.model = model
        self.max_recursive_depth = max_recursive_depth


    bar = """
                # function under test <bar>
                a = 1
                def bar(b):
                  a = 2
                  return b
                """
    
    # Cache of functions we have just explored TODO eventually this might be global for every run of the system everywhere
    function_cache = {}
    def _get_function_within_package(self,function_name,depth):
        if function_name not in self.function_cache:
           self.function_cache[function_name] =self.write_spec(function_name,self.bar,depth)
        else:
          print("CACHE HIT FOR (package): ",function_name)

        return self.function_cache[function_name]
    
    def _get_function_within_code(self,function_name,depth):
        if function_name not in self.function_cache:
          self.function_cache[function_name] =self.write_spec(function_name,self.bar,depth)
        else:
           print("CACHE HIT FOR (code): ",function_name)
        return self.function_cache[function_name]

    def write_spec(self,func_name,function_text,depth=0):
        """
        Given the function_text a textfile including the function and its surronding scope and the name of which function is to be found
        Returns a JSON object (python dict) 
        """
        print("WRITE SPEC CALLED FOR: ",func_name, "WITH DEPTH: ",depth)
        logger.info("WRITE SPEC CALLED FOR: "+func_name+" WITH DEPTH: "+str(depth))

        # Stop at max depth to prevent infinite loops
        if depth >= self.max_recursive_depth:
           return "Max recursive depth reached, bubble this error up."
        

        still_running = True
        messages = [
                    {"role": "system", "content": self.system_message},
                    {"role": "user", "content": function_text},
                ]
        
        while still_running:
          logger.info("CHAT REQ SENT")
          res = client.chat.completions.create(
                  model=self.model,
                  # response_format={"type": "json_object"},
                  messages=messages,
                  tools=self.tools,
                  tool_choice='auto',
                  temperature=0.1
          )
          logger.info("CHAT REQ END")

          response_message = res.choices[0].message
          tool_calls = response_message.tool_calls
          if tool_calls:
            messages.append(response_message)
            available_functions = {
                "get_function_within_code": self._get_function_within_code,
                "get_function_within_package": self._get_function_within_package,
            }
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                # If the system hallucinates a tool call inform it
                if function_name not in available_functions:
                   messages.append({
                        "tool_call_id": tool_call.id, 
                        "role": "tool", # Indicates this message is from tool use
                        "name": function_name,
                        "content": "This is not an available tool you must do this work yourself.",
                    })
                else:
                  function_to_call = available_functions[function_name]
                  function_args = json.loads(tool_call.function.arguments)
                  # Call the tool and get the response
                  function_response = function_to_call(
                      function_name=function_args.get("function_name"),
                      depth=depth+1
                  )
                  print(function_response)
                  messages.append({
                          "tool_call_id": tool_call.id, 
                          "role": "tool", # Indicates this message is from tool use
                          "name": function_name,
                          "content": function_response,
                      })
          else:
             still_running = False

        print("RESULT OF WRITE SPEC CALLED FOR: ",func_name, "WITH DEPTH: ",depth)
        logger.info(repr(res.choices[0].message.content))
        logger.info("FINISHED WRITE SPEC CALLED FOR: "+func_name+" WITH DEPTH: "+str(depth))
        print(res.choices[0].message.content)
        return (res.choices[0].message.content)
                
          



In [165]:
from urllib.parse import quote
import requests 
from bs4 import BeautifulSoup

HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}

def get_links_of_top_n_google_results(query,n=3):

    res = requests.get(
        url="https://www.google.com/search",
    headers=HEADERS,
    params={
        "q": query,
        'num':n,
    },)

    soup = BeautifulSoup(res.text, "html.parser")
    output = []
    result_block = soup.find_all("div", attrs={"class": "g"})
    for result in result_block:
        try:
            output.append(result.find("a",href=True)['href'])
        except:
            print("Failed to find google search link")

    return output
def get_text_of_top_n_google_results(query,n=3):
    links = get_links_of_top_n_google_results(query,n)
    # Free service to convert website into ai readable text
    jina_prefix = "https://r.jina.ai"
    output = ""
    for link in links:
        try:
            url = jina_prefix+"/"+link
            res = requests.get(url=url,headers=HEADERS)
            output += res.text
        except:
            print(f"Failed to get text for {url}")
            
            
    return output

   

get_text_of_top_n_google_results("np.argmax",1)

'Title: numpy.argmax — NumPy v2.0 Manual\n\nURL Source: https://numpy.org/doc/2.0/reference/generated/numpy.argmax.html\n\nMarkdown Content:\nnumpy.argmax — NumPy v2.0 Manual\n===============        \n\n[Skip to main content](https://numpy.org/doc/2.0/reference/generated/numpy.argmax.html#main-content)\n\nBack to top  \n\n Ctrl+K\n\n [![Image 1: NumPy v2.0 Manual - Home](https://numpy.org/doc/2.0/_static/numpylogo.svg) ![Image 2: NumPy v2.0 Manual - Home](https://numpy.org/doc/2.0/_static/numpylogo_dark.svg)](https://numpy.org/doc/2.0/index.html)\n\n*   [User Guide](https://numpy.org/doc/2.0/user/index.html)\n*   [API reference](https://numpy.org/doc/2.0/reference/index.html)\n*   [Building from source](https://numpy.org/doc/2.0/building/index.html)\n*   [Development](https://numpy.org/doc/2.0/dev/index.html)\n*   [Release notes](https://numpy.org/doc/2.0/release.html)\n*   [Learn](https://numpy.org/numpy-tutorials/)\n*   More\n    *   [NEPs](https://numpy.org/neps)\n\nChoose version\n

In [171]:
# TODO make analyze_function tool that gest called to write the spec, call it after all other stuff is done

f = open("system_messages/chatgpt3.txt","r")
system_message = f.read()
python_code = """
                # function under test <foo>|
                a = 0
                def foo(b):
                    a = b + a
                    b = bar(b)
                    return b


            
"""

model = 'llama3-groq-70b-8192-tool-use-preview'
# model = "mixtral-8x7b-32768"

writer = FunctionSpecWriter(system_message,model)
res = writer.write_spec("foo",python_code)

WRITE SPEC CALLED FOR:  foo WITH DEPTH:  0
WRITE SPEC CALLED FOR:  bar WITH DEPTH:  1
RESULT OF WRITE SPEC CALLED FOR:  bar WITH DEPTH:  1
  "function_name": "bar"
  "inputs": [
    {
      "name": "b",
      "type": "int",
      "domain": "Any integer value"
    }
  ],
  "return": [
    {
      "type": "int",
      "domain": "The value of the input parameter b"
    }
  ],
  "side_effects": [
    {
      "description": "Changes the value of the global variable a",
      "condition": "",
      "origin": "a = 2 in the function"
    }
  ],
  "dependencies": []
}
  "function_name": "bar"
  "inputs": [
    {
      "name": "b",
      "type": "int",
      "domain": "Any integer value"
    }
  ],
  "return": [
    {
      "type": "int",
      "domain": "The value of the input parameter b"
    }
  ],
  "side_effects": [
    {
      "description": "Changes the value of the global variable a",
      "condition": "",
      "origin": "a = 2 in the function"
    }
  ],
  "dependencies": []
}
CACHE H

KeyboardInterrupt: 

In [162]:
print(res)

Based on the analysis, the function `foo` has the following specifications:

- **Function Name:** `foo`
- **Inputs:** 
  - `b`: An integer value
- **Return:** 
  - An integer value, which is the modified value of `b`
- **Side Effects:**
  - Modifies the local variable `a` by adding `b` to it
  - Modifies the local variable `a` by setting it to `2`
- **Dependencies:** None

The function `bar` has been analyzed as well, and it modifies the local variable `a` and returns the value of the input parameter `b`.


In [152]:
res

'Based on the results from the tools, the function `bar` modifies the value of the global variable `a` and returns the value of the input parameter `b`. It has no dependencies. Now that we have the function specs for `bar`, we can proceed with analyzing the function `foo`.'

hi
